In [1]:
import pandas as pd
import numpy as np


# for Modelling
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, GRU, LSTM, Dense

from tensorflow.keras.models import load_model

# for Evaluation
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.metrics import mean_absolute_error, r2_score


In [2]:
df= pd.read_csv('ay_t1.csv')
df

,Time id,Material,Quantity,Total_Value,Material_Id
0,1,C87-200477,3.0,649324.24,4774
1,1,S01-070036,2.0,112000.00,6154
2,1,S07-105060,3.0,82500.00,6718
3,1,S12-060177,35.0,144025.52,7852
4,1,S12-160158,14.0,349622.89,7924
...,...,...,...,...,...
449,38,S21-040025-06,2.0,751296.57,8420
450,38,S22-080093,96.0,85344.15,8522
451,38,S25-010204-01,10.0,369925.44,8994
452,39,S12-060177,1.0,4110.60,7852


In [3]:
import pandas as pd

# Assuming your DataFrame is named df

# Filter the DataFrame for Time ids 37 and 38
filtered_df = df[df['Time id'].isin([37, 38])]

# Display the resulting DataFrame
print(filtered_df)


     Time id          Material  Quantity  Total_Value  Material_Id
427       37        C87-200477       4.0   1205871.69         4774
428       37        S07-105060      10.0    302600.00         6718
429       37        S12-060177      54.0    221972.50         7852
430       37        S12-160158      11.0    310020.07         7924
431       37        S12-160210      12.0    236200.68         7930
432       37        S12-160342       6.0    131076.15        33547
433       37     S21-010399-06       6.0   2166096.29         8387
434       37        S21-040014       4.0    269612.00         8416
435       37     S21-040025-06       6.0   2247282.14         8420
436       37        S22-080093     124.0    111267.65         8522
437       37     S25-010204-01       6.0    221538.10         8994
438       38  A30-14S-0139-SET       3.0    215887.50        24712
439       38        C87-200477       2.0    599020.76         4774
440       38        S01-070036       4.0    246400.00         

In [4]:
from sklearn.metrics import mean_absolute_error, r2_score
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import SimpleRNN, GRU, LSTM, Dense
import numpy as np
import pandas as pd

# Define the evaluate_model function with MAPE
def evaluate_model(model, X_test, y_test):
    y_pred = model.predict(X_test)
    mae = mean_absolute_error(y_test, y_pred)
    mape = np.mean(np.abs((y_test - y_pred) / y_test)) * 100
    r2 = r2_score(y_test, y_pred)
    return mae, mape, r2

# Define train and test periods
train_start_id = 25
train_end_id = 36
forecast_time_id = 37

# Create an empty DataFrame to store forecasted values
forecast_df = pd.DataFrame(columns=['Time id', 'Material', 'Quantity', 'Total_Value', 'Material_Id', 'Model', 'Forecasted', 'MAE', 'MAPE', 'R2'])

# Define a function to build and train RNN model
def build_and_train_model(X_train, y_train, model_type):
    model = Sequential()
    if model_type == 'SimpleRNN':
        model.add(SimpleRNN(units=64, input_shape=(X_train.shape[1], X_train.shape[2])))
    elif model_type == 'GRU':
        model.add(GRU(units=64, input_shape=(X_train.shape[1], X_train.shape[2])))
    elif model_type == 'LSTM':
        model.add(LSTM(units=64, input_shape=(X_train.shape[1], X_train.shape[2])))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mse')
    model.fit(X_train, y_train, epochs=100, batch_size=32, verbose=0)
    return model

# Iterate over each material id
for material_id in df['Material_Id'].unique():
    print(f"Evaluating models for Material ID: {material_id}")
    
    # Filter data for the current material id
    material_data = df[df['Material_Id'] == material_id]
    
    # Filter training data
    # Filter training data
    train_data = material_data[(material_data['Time id'] >= train_start_id) & (material_data['Time id'] <= train_end_id)]


    # Skip evaluation if there is no training data
    if train_data.empty:
        print(f"No training data available for Material ID: {material_id}. Skipping evaluation.")
        continue

    # Scale the data
    scaler = MinMaxScaler()
    train_scaled = scaler.fit_transform(train_data[['Quantity']])

    # Reshape input to be [samples, time steps, features]
    X_train = np.reshape(train_scaled, (train_scaled.shape[0], 1, 1))
    y_train = train_scaled[:, 0]  # Target variable is Quantity

    # Evaluate SimpleRNN, GRU, and LSTM models
    for model_type in ['SimpleRNN', 'GRU', 'LSTM']:
        print(f"Training {model_type} model for Material ID: {material_id}")
        
        # Build and train the model
        model = build_and_train_model(X_train, y_train, model_type)
        
        # Forecast for the next time id (Time id 37)
        forecasted_values = model.predict(X_train[-1:])  # Use the last time step of the training data for forecasting
        forecasted_values = scaler.inverse_transform(forecasted_values.reshape(-1, 1)).flatten()
        forecast = forecasted_values[0]

        # Evaluate the model for the current Material Id
        mae, mape, r2 = evaluate_model(model, X_train, y_train)
        print(f"MAE: {mae}, MAPE: {mape}, R-squared: {r2}")

        # Append to forecast DataFrame
        forecast_df = forecast_df.append({
            'Time id': forecast_time_id,
            'Material': material_data['Material'].iloc[-1],
            'Quantity': material_data['Quantity'].iloc[-1],
            'Total_Value': material_data['Total_Value'].iloc[-1],
            'Material_Id': material_id,
            'Model': model_type,
            'Forecasted': forecast,
            'MAE': mae,
            'MAPE': mape,
            'R2': r2
        }, ignore_index=True)

# Display the forecast DataFrame
print(forecast_df)


Evaluating models for Material ID: 4774
Training SimpleRNN model for Material ID: 4774


C:\Users\sohan\anaconda3\lib\site-packages\keras\src\layers\rnn\rnn.py:205: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step
MAE: 0.04189224503558087, MAPE: inf, R-squared: 0.9744007195902851
Training GRU model for Material ID: 4774


C:\Users\sohan\AppData\Local\Temp\ipykernel_16196\4108763989.py:12: RuntimeWarning: divide by zero encountered in divide
  mape = np.mean(np.abs((y_test - y_pred) / y_test)) * 100
C:\Users\sohan\AppData\Local\Temp\ipykernel_16196\4108763989.py:80: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  forecast_df = forecast_df.append({
C:\Users\sohan\anaconda3\lib\site-packages\keras\src\layers\rnn\rnn.py:205: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 237ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 265ms/step
MAE: 0.12740628857805267, MAPE: inf, R-squared: 0.7952123809450723
Training LSTM model for Material ID: 4774


...

C:\Users\sohan\AppData\Local\Temp\ipykernel_16196\4108763989.py:12: RuntimeWarning: divide by zero encountered in divide
  mape = np.mean(np.abs((y_test - y_pred) / y_test)) * 100
C:\Users\sohan\AppData\Local\Temp\ipykernel_16196\4108763989.py:80: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  forecast_df = forecast_df.append({
C:\Users\sohan\anaconda3\lib\site-packages\keras\src\layers\rnn\rnn.py:205: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 319ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 343ms/step
MAE: 0.17968846956888834, MAPE: inf, R-squared: 0.7117981115307916
Training LSTM model for Material ID: 33547


C:\Users\sohan\AppData\Local\Temp\ipykernel_16196\4108763989.py:12: RuntimeWarning: divide by zero encountered in divide
  mape = np.mean(np.abs((y_test - y_pred) / y_test)) * 100
C:\Users\sohan\AppData\Local\Temp\ipykernel_16196\4108763989.py:80: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  forecast_df = forecast_df.append({
C:\Users\sohan\anaconda3\lib\site-packages\keras\src\layers\rnn\rnn.py:205: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 271ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 289ms/step
MAE: 0.18367906312147775, MAPE: inf, R-squared: 0.7121162960858898
   Time id          Material  Quantity  Total_Value Material_Id      Model  \
0       37        C87-200477       2.0    599020.76        4774  SimpleRNN   
1       37        C87-200477       2.0    599020.76        4774        GRU   
2       37        C87-200477       2.0    599020.76        4774       LSTM   
3       37        S01-070036       4.0    246400.00        6154  SimpleRNN   
4       37        S01-070036       4.0    246400.00        6154        GRU   
5       37        S01-070036       4.0    246400.00        6154       LSTM   
6       37        S07-105060       8.0    242080.00        6718  SimpleRNN   
7       37        S07-105060       8.0    242080.00        6718        GRU   
8       37        S07-105060       8.0    242080.00        6718       LSTM   
9       37        S12-060177       1.0      4110.60        7852  SimpleRNN 

C:\Users\sohan\AppData\Local\Temp\ipykernel_16196\4108763989.py:12: RuntimeWarning: divide by zero encountered in divide
  mape = np.mean(np.abs((y_test - y_pred) / y_test)) * 100
C:\Users\sohan\AppData\Local\Temp\ipykernel_16196\4108763989.py:80: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  forecast_df = forecast_df.append({


In [6]:
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import SimpleRNN, GRU, LSTM, Dense
import numpy as np
import pandas as pd

# Define the evaluate_model function with MAPE
def evaluate_model(y_pred, y_actual):
    mape = np.mean(np.abs((y_actual - y_pred) / y_actual)) * 100
    return mape

# Define train and test periods
train_start_id = 25
train_end_id = 36
forecast_time_id = 37

# Create an empty DataFrame to store forecasted values
forecast_df = pd.DataFrame(columns=['Material_Id', 'Model', 'MAPE'])

# Define a function to build and train RNN model
def build_and_train_model(X_train, y_train, model_type):
    model = Sequential()
    if model_type == 'SimpleRNN':
        model.add(SimpleRNN(units=64, input_shape=(X_train.shape[1], X_train.shape[2])))
    elif model_type == 'GRU':
        model.add(GRU(units=64, input_shape=(X_train.shape[1], X_train.shape[2])))
    elif model_type == 'LSTM':
        model.add(LSTM(units=64, input_shape=(X_train.shape[1], X_train.shape[2])))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mse')
    model.fit(X_train, y_train, epochs=100, batch_size=32, verbose=0)
    return model

# Iterate over each material id
for material_id in df['Material_Id'].unique():
    print(f"Evaluating models for Material ID: {material_id}")
    
    # Filter data for the current material id
    material_data = df[df['Material_Id'] == material_id]
    
    # Filter training data
    train_data = material_data[(material_data['Time id'] >= train_start_id) & (material_data['Time id'] <= train_end_id)]
    test_data = material_data[material_data['Time id'] == forecast_time_id]

    # Skip evaluation if there is no training or test data
    if train_data.empty or test_data.empty:
        print(f"No training or test data available for Material ID: {material_id}. Skipping evaluation.")
        continue

    # Scale the data
    scaler = MinMaxScaler()
    train_scaled = scaler.fit_transform(train_data[['Quantity']])

    # Reshape input to be [samples, time steps, features]
    X_train = np.reshape(train_scaled, (train_scaled.shape[0], 1, 1))
    y_train = train_scaled[:, 0]  # Target variable is Quantity

    # Evaluate SimpleRNN, GRU, and LSTM models
    for model_type in ['SimpleRNN', 'GRU', 'LSTM']:
        print(f"Evaluating {model_type} model for Material ID: {material_id}")

        # Build and train the model
        model = build_and_train_model(X_train, y_train, model_type)

        # Forecast for the next time id (Time id 37)
        forecast_scaled = model.predict(X_train[-1:])  # Use the last time step of the training data for forecasting
        forecast = scaler.inverse_transform(forecast_scaled.reshape(-1, 1)).flatten()[0]

        # Get actual Quantity value for Time id 37
        actual_quantity = test_data['Quantity'].values[0]

        # Evaluate the model for the current Material Id
        mape = evaluate_model(np.array([forecast]), np.array([actual_quantity]))
        print(f"MAPE: {mape}")

        # Append to forecast DataFrame
        forecast_df = forecast_df.append({
            'Material_Id': material_id,
            'Model': model_type,
            'MAPE': mape
        }, ignore_index=True)

# Display the forecast DataFrame with MAPE values
print(forecast_df)


Evaluating models for Material ID: 4774
Evaluating SimpleRNN model for Material ID: 4774


C:\Users\sohan\anaconda3\lib\site-packages\keras\src\layers\rnn\rnn.py:205: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 249ms/step
MAPE: 65.08293151855469
Evaluating GRU model for Material ID: 4774


C:\Users\sohan\AppData\Local\Temp\ipykernel_16196\472838995.py:77: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  forecast_df = forecast_df.append({
C:\Users\sohan\anaconda3\lib\site-packages\keras\src\layers\rnn\rnn.py:205: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 346ms/step
MAPE: 64.34310674667358
Evaluating LSTM model for Material ID: 4774


...

C:\Users\sohan\AppData\Local\Temp\ipykernel_16196\472838995.py:77: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  forecast_df = forecast_df.append({
C:\Users\sohan\anaconda3\lib\site-packages\keras\src\layers\rnn\rnn.py:205: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 311ms/step
MAPE: 71.92049026489258
Evaluating LSTM model for Material ID: 33547


C:\Users\sohan\AppData\Local\Temp\ipykernel_16196\472838995.py:77: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  forecast_df = forecast_df.append({
C:\Users\sohan\anaconda3\lib\site-packages\keras\src\layers\rnn\rnn.py:205: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 267ms/step
MAPE: 88.26258977254233
   Material_Id      Model        MAPE
0         4774  SimpleRNN   65.082932
1         4774        GRU   64.343107
2         4774       LSTM   60.118210
3         6718  SimpleRNN   22.760534
4         6718        GRU   26.214862
5         6718       LSTM   28.791089
6         7852  SimpleRNN   34.253241
7         7852        GRU   27.628821
8         7852       LSTM   20.046404
9         7924  SimpleRNN   12.228862
10        7924        GRU   41.505883
11        7924       LSTM   45.208749
12        7930  SimpleRNN  135.063601
13        7930        GRU  190.125243
14        7930       LSTM  208.928680
15        8387  SimpleRNN   83.987324
16        8387        GRU   42.737357
17        8387       LSTM   34.724585
18        8416  SimpleRNN   24.599701
19        8416        GRU   25.064057
20        8416       LSTM   24.437332
21        8420  SimpleRNN    5.251416
22        8420        GRU    7.692893
23        8420       LSTM

C:\Users\sohan\AppData\Local\Temp\ipykernel_16196\472838995.py:77: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  forecast_df = forecast_df.append({


In [7]:
forecast_df.to_csv('ay2.csv', index=False)


In [8]:
from sklearn.metrics import mean_absolute_error, r2_score
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import SimpleRNN, GRU, LSTM, Dense
import numpy as np
import pandas as pd

# Define the evaluate_model function with MAPE
def evaluate_model(model, X_test, y_test):
    y_pred = model.predict(X_test)
    mae = mean_absolute_error(y_test, y_pred)
    mape = np.mean(np.abs((y_test - y_pred) / y_test)) * 100
    r2 = r2_score(y_test, y_pred)
    return mae, mape, r2

# Define train and test periods
train_start_id = 25
train_end_id = 37
forecast_time_id = 38

# Create an empty DataFrame to store forecasted values
forecast_df = pd.DataFrame(columns=['Time id', 'Material', 'Quantity', 'Total_Value', 'Material_Id', 'Model', 'Forecasted', 'MAE', 'MAPE', 'R2'])

# Define a function to build and train RNN model
def build_and_train_model(X_train, y_train, model_type):
    model = Sequential()
    if model_type == 'SimpleRNN':
        model.add(SimpleRNN(units=64, input_shape=(X_train.shape[1], X_train.shape[2])))
    elif model_type == 'GRU':
        model.add(GRU(units=64, input_shape=(X_train.shape[1], X_train.shape[2])))
    elif model_type == 'LSTM':
        model.add(LSTM(units=64, input_shape=(X_train.shape[1], X_train.shape[2])))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mse')
    model.fit(X_train, y_train, epochs=100, batch_size=32, verbose=0)
    return model

# Iterate over each material id
for material_id in df['Material_Id'].unique():
    print(f"Evaluating models for Material ID: {material_id}")
    
    # Filter data for the current material id
    material_data = df[df['Material_Id'] == material_id]
    
    # Filter training data
    # Filter training data
    train_data = material_data[(material_data['Time id'] >= train_start_id) & (material_data['Time id'] <= train_end_id)]


    # Skip evaluation if there is no training data
    if train_data.empty:
        print(f"No training data available for Material ID: {material_id}. Skipping evaluation.")
        continue

    # Scale the data
    scaler = MinMaxScaler()
    train_scaled = scaler.fit_transform(train_data[['Quantity']])

    # Reshape input to be [samples, time steps, features]
    X_train = np.reshape(train_scaled, (train_scaled.shape[0], 1, 1))
    y_train = train_scaled[:, 0]  # Target variable is Quantity

    # Evaluate SimpleRNN, GRU, and LSTM models
    for model_type in ['SimpleRNN', 'GRU', 'LSTM']:
        print(f"Training {model_type} model for Material ID: {material_id}")
        
        # Build and train the model
        model = build_and_train_model(X_train, y_train, model_type)
        
        # Forecast for the next time id (Time id 37)
        forecasted_values = model.predict(X_train[-1:])  # Use the last time step of the training data for forecasting
        forecasted_values = scaler.inverse_transform(forecasted_values.reshape(-1, 1)).flatten()
        forecast = forecasted_values[0]

        # Evaluate the model for the current Material Id
        mae, mape, r2 = evaluate_model(model, X_train, y_train)
        print(f"MAE: {mae}, MAPE: {mape}, R-squared: {r2}")

        # Append to forecast DataFrame
        forecast_df = forecast_df.append({
            'Time id': forecast_time_id,
            'Material': material_data['Material'].iloc[-1],
            'Quantity': material_data['Quantity'].iloc[-1],
            'Total_Value': material_data['Total_Value'].iloc[-1],
            'Material_Id': material_id,
            'Model': model_type,
            'Forecasted': forecast,
            'MAE': mae,
            'MAPE': mape,
            'R2': r2
        }, ignore_index=True)

# Display the forecast DataFrame
print(forecast_df)


Evaluating models for Material ID: 4774
Training SimpleRNN model for Material ID: 4774


C:\Users\sohan\anaconda3\lib\site-packages\keras\src\layers\rnn\rnn.py:205: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 222ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 235ms/step
MAE: 0.10813493695523997, MAPE: inf, R-squared: 0.8390030529398969
Training GRU model for Material ID: 4774


C:\Users\sohan\AppData\Local\Temp\ipykernel_16196\3232560010.py:12: RuntimeWarning: divide by zero encountered in divide
  mape = np.mean(np.abs((y_test - y_pred) / y_test)) * 100
C:\Users\sohan\AppData\Local\Temp\ipykernel_16196\3232560010.py:80: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  forecast_df = forecast_df.append({
C:\Users\sohan\anaconda3\lib\site-packages\keras\src\layers\rnn\rnn.py:205: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 314ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 367ms/step
MAE: 0.11681034208999734, MAPE: inf, R-squared: 0.8200453124930204
Training LSTM model for Material ID: 4774


...

C:\Users\sohan\AppData\Local\Temp\ipykernel_16196\3232560010.py:12: RuntimeWarning: divide by zero encountered in divide
  mape = np.mean(np.abs((y_test - y_pred) / y_test)) * 100
C:\Users\sohan\AppData\Local\Temp\ipykernel_16196\3232560010.py:80: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  forecast_df = forecast_df.append({
C:\Users\sohan\anaconda3\lib\site-packages\keras\src\layers\rnn\rnn.py:205: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 869ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 325ms/step
MAE: 0.1552011566330688, MAPE: inf, R-squared: 0.7683858388093343
Training LSTM model for Material ID: 33547


C:\Users\sohan\AppData\Local\Temp\ipykernel_16196\3232560010.py:12: RuntimeWarning: divide by zero encountered in divide
  mape = np.mean(np.abs((y_test - y_pred) / y_test)) * 100
C:\Users\sohan\AppData\Local\Temp\ipykernel_16196\3232560010.py:80: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  forecast_df = forecast_df.append({
C:\Users\sohan\anaconda3\lib\site-packages\keras\src\layers\rnn\rnn.py:205: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 288ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 294ms/step
MAE: 0.2032910239214849, MAPE: inf, R-squared: 0.6211497074136456
   Time id          Material  Quantity  Total_Value Material_Id      Model  \
0       38        C87-200477       2.0    599020.76        4774  SimpleRNN   
1       38        C87-200477       2.0    599020.76        4774        GRU   
2       38        C87-200477       2.0    599020.76        4774       LSTM   
3       38        S01-070036       4.0    246400.00        6154  SimpleRNN   
4       38        S01-070036       4.0    246400.00        6154        GRU   
5       38        S01-070036       4.0    246400.00        6154       LSTM   
6       38        S07-105060       8.0    242080.00        6718  SimpleRNN   
7       38        S07-105060       8.0    242080.00        6718        GRU   
8       38        S07-105060       8.0    242080.00        6718       LSTM   
9       38        S12-060177       1.0      4110.60        7852  SimpleRNN  

C:\Users\sohan\AppData\Local\Temp\ipykernel_16196\3232560010.py:12: RuntimeWarning: divide by zero encountered in divide
  mape = np.mean(np.abs((y_test - y_pred) / y_test)) * 100
C:\Users\sohan\AppData\Local\Temp\ipykernel_16196\3232560010.py:80: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  forecast_df = forecast_df.append({


In [9]:
forecast_df.to_csv('ay3.csv', index=False)


In [10]:
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import SimpleRNN, GRU, LSTM, Dense
import numpy as np
import pandas as pd

# Define the evaluate_model function with MAPE
def evaluate_model(y_pred, y_actual):
    mape = np.mean(np.abs((y_actual - y_pred) / y_actual)) * 100
    return mape

# Define train and test periods
train_start_id = 25
train_end_id = 37
forecast_time_id = 38

# Create an empty DataFrame to store forecasted values
forecast_df = pd.DataFrame(columns=['Material_Id', 'Model', 'MAPE'])

# Define a function to build and train RNN model
def build_and_train_model(X_train, y_train, model_type):
    model = Sequential()
    if model_type == 'SimpleRNN':
        model.add(SimpleRNN(units=64, input_shape=(X_train.shape[1], X_train.shape[2])))
    elif model_type == 'GRU':
        model.add(GRU(units=64, input_shape=(X_train.shape[1], X_train.shape[2])))
    elif model_type == 'LSTM':
        model.add(LSTM(units=64, input_shape=(X_train.shape[1], X_train.shape[2])))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mse')
    model.fit(X_train, y_train, epochs=100, batch_size=32, verbose=0)
    return model

# Iterate over each material id
for material_id in df['Material_Id'].unique():
    print(f"Evaluating models for Material ID: {material_id}")
    
    # Filter data for the current material id
    material_data = df[df['Material_Id'] == material_id]
    
    # Filter training data
    train_data = material_data[(material_data['Time id'] >= train_start_id) & (material_data['Time id'] <= train_end_id)]
    test_data = material_data[material_data['Time id'] == forecast_time_id]

    # Skip evaluation if there is no training or test data
    if train_data.empty or test_data.empty:
        print(f"No training or test data available for Material ID: {material_id}. Skipping evaluation.")
        continue

    # Scale the data
    scaler = MinMaxScaler()
    train_scaled = scaler.fit_transform(train_data[['Quantity']])

    # Reshape input to be [samples, time steps, features]
    X_train = np.reshape(train_scaled, (train_scaled.shape[0], 1, 1))
    y_train = train_scaled[:, 0]  # Target variable is Quantity

    # Evaluate SimpleRNN, GRU, and LSTM models
    for model_type in ['SimpleRNN', 'GRU', 'LSTM']:
        print(f"Evaluating {model_type} model for Material ID: {material_id}")

        # Build and train the model
        model = build_and_train_model(X_train, y_train, model_type)

        # Forecast for the next time id (Time id 37)
        forecast_scaled = model.predict(X_train[-1:])  # Use the last time step of the training data for forecasting
        forecast = scaler.inverse_transform(forecast_scaled.reshape(-1, 1)).flatten()[0]

        # Get actual Quantity value for Time id 37
        actual_quantity = test_data['Quantity'].values[0]

        # Evaluate the model for the current Material Id
        mape = evaluate_model(np.array([forecast]), np.array([actual_quantity]))
        print(f"MAPE: {mape}")

        # Append to forecast DataFrame
        forecast_df = forecast_df.append({
            'Material_Id': material_id,
            'Model': model_type,
            'MAPE': mape
        }, ignore_index=True)

# Display the forecast DataFrame with MAPE values
print(forecast_df)


Evaluating models for Material ID: 4774
Evaluating SimpleRNN model for Material ID: 4774


C:\Users\sohan\anaconda3\lib\site-packages\keras\src\layers\rnn\rnn.py:205: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 221ms/step
MAPE: 135.22837162017822
Evaluating GRU model for Material ID: 4774


C:\Users\sohan\AppData\Local\Temp\ipykernel_16196\1359943296.py:77: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  forecast_df = forecast_df.append({
C:\Users\sohan\anaconda3\lib\site-packages\keras\src\layers\rnn\rnn.py:205: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 304ms/step
MAPE: 137.1208667755127
Evaluating LSTM model for Material ID: 4774


...

C:\Users\sohan\AppData\Local\Temp\ipykernel_16196\1359943296.py:77: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  forecast_df = forecast_df.append({
C:\Users\sohan\anaconda3\lib\site-packages\keras\src\layers\rnn\rnn.py:205: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 286ms/step
MAPE: 161.9815508524577
Evaluating LSTM model for Material ID: 33547


C:\Users\sohan\AppData\Local\Temp\ipykernel_16196\1359943296.py:77: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  forecast_df = forecast_df.append({
C:\Users\sohan\anaconda3\lib\site-packages\keras\src\layers\rnn\rnn.py:205: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 246ms/step
MAPE: 225.33130645751953
   Material_Id      Model        MAPE
0         4774  SimpleRNN  135.228372
1         4774        GRU  137.120867
2         4774       LSTM  154.629111
3         6154  SimpleRNN   63.007998
4         6154        GRU   42.205137
5         6154       LSTM   18.028736
6         6718  SimpleRNN   21.801734
7         6718        GRU    6.109333
8         6718       LSTM    0.938827
9         7852  SimpleRNN   14.932585
10        7852        GRU   13.149684
11        7852       LSTM   11.062916
12        7924  SimpleRNN   31.738424
13        7924        GRU   19.017696
14        7924       LSTM   12.835121
15        7930  SimpleRNN   15.879210
16        7930        GRU    8.894070
17        7930       LSTM   28.983792
18        8387  SimpleRNN   48.774838
19        8387        GRU   45.248735
20        8387       LSTM   47.239709
21        8416  SimpleRNN   96.672308
22        8416        GRU   75.055766
23        8416       LST

C:\Users\sohan\AppData\Local\Temp\ipykernel_16196\1359943296.py:77: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  forecast_df = forecast_df.append({


In [11]:
forecast_df.to_csv('ay4.csv', index=False)
